Install packages

In [2]:
!pip install pandas numpy tensorflow matplotlib keras_tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 163.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 194.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 213.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.3 MB/s eta 0:00:00


Import packages

In [3]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from tensorflow import keras
import keras_tuner as kt

Import datasets

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sujaymann/handwritten-english-characters-and-digits")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'handwritten-english-characters-and-digits' dataset.
Path to dataset files: /kaggle/input/handwritten-english-characters-and-digits


In [5]:
import os
print("Files in dataset folder:")
print(os.listdir(path))

Files in dataset folder:
['handwritten-english-characters-and-digits', 'image_labels.csv', 'augmented_images']


In [6]:
train_dir = os.path.join(path, "handwritten-english-characters-and-digits/combined_folder/train") #'/kaggle/input/handwritten-english-characters-and-digits/handwritten-english-characters-and-digits/combined_folder/train'
test_dir  = os.path.join(path, "handwritten-english-characters-and-digits/combined_folder/test") #'/kaggle/input/handwritten-english-characters-and-digits/handwritten-english-characters-and-digits/combined_folder/test'
augmented_data= os.path.join(path, "augmented_images/augmented_images1") #'/kaggle/input/handwritten-english-characters-and-digits/augmented_images/augmented_images1'

Data preprocessing

In [7]:
validate_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,              # root directory containing one subfolder per class
    image_size=(128, 128),  # resize every image to 128x128 (bilinear); model input size
    batch_size=32,          # number of samples per batch
    label_mode='categorical'# return one-hot encoded labels (shape: [batch, num_classes])
)

Found 2728 files belonging to 62 classes.


In [8]:
augmented_ds = tf.keras.utils.image_dataset_from_directory(
    augmented_data,          # root folder containing per-class subdirectories
    image_size=(128, 128),   # resize every image to 128x128 (bilinear)
    batch_size=32,           # number of samples per batch
    label_mode='categorical' # return one-hot encoded labels (shape: [B, num_classes])
)

Found 13640 files belonging to 62 classes.


In [9]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,                # root directory of the test set
    image_size=(128, 128),   # resize all images to 128x128 (bilinear)
    batch_size=32,           # batch size for evaluation
    label_mode='categorical',# one-hot encoded labels (shape: [B, num_classes])
)

Found 682 files belonging to 62 classes.


Train CNN Model with best selected parameters

In [11]:
# Sequential CNN for 62-class handwritten character recognition.
# Design choices:
# - 128x128 inputs to preserve thin strokes.
# - 3x3 convs + BatchNorm + ReLU for stable, efficient feature extraction.
# - Progressive channel widening (32→512) to learn richer features at coarser scales.
# - MaxPooling to downsample; Dropout increases with depth to combat overfitting.
# - GlobalAveragePooling avoids huge dense layers and improves generalisation.

model = models.Sequential([
    # ---- Input & normalization ----
    layers.Input(shape=(128, 128, 3)),
    layers.Rescaling(1./255),              # Scale pixels from [0,255] → [0,1]

    # ---- Block 1: low-level edges/strokes ----
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),  # edge detectors
    layers.BatchNormalization(),                                    # stabilise activations
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),  # richer low-level features
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),                          # 128→64
    layers.Dropout(0.2),                                            # mild regularisation

    # ---- Block 2: mid-level motifs (corners, junctions) ----
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),                          # 64→32
    layers.Dropout(0.25),

    # ---- Block 3: character parts (loops, crossbars) ----
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),                          # 32→16
    layers.Dropout(0.3),

    # ---- Block 4: higher-level shapes ----
    layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),                          # 16→8
    layers.Dropout(0.35),

    # ---- Block 5: optional deep features (useful for hard pairs like O/0, l/1) ----
    layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),                          # 8→4
    layers.Dropout(0.4),

    # ---- Classification head ----
    layers.GlobalAveragePooling2D(),                                # pool per-channel means
    layers.Dense(512, activation='relu'),                           # compact classifier
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(62, activation='softmax')                          # 62-class probabilities
])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,016,670 (19.14 MB)

 Trainable params: 5,011,678 (19.12 MB)

 Non-trainable params: 4,992 (19.50 KB)

In [13]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),                                # lr = 0.0001
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),# label_smoothing = 0.05
    metrics=['accuracy']
)

# NOTE on batch size = 64:
# Batch size is set when you build the datasets, not in model.fit().
# Rebuild datasets with batch_size=64 (or re-batch) to match the best params, e.g.:
# augmented_ds = augmented_ds.unbatch().batch(64).prefetch(tf.data.AUTOTUNE)
# validate_ds  = validate_ds.unbatch().batch(64).prefetch(tf.data.AUTOTUNE)

# NOTE on augment=True:
# Ensure your training pipeline applies augmentation (e.g., preprocessing layers in the model
# or ds.map(augment_fn)). Here we assume `augmented_ds` already contains augmented samples.

# --- Train with early stopping & LR scheduling ---
history = model.fit(
    augmented_ds,                 # augmented training data (augment=True)
    validation_data=validate_ds,  # clean validation split
    epochs=40,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy", mode="max", patience=10, restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", mode="min", factor=0.5, patience=5, min_lr=1e-7
        )
    ]
)


Epoch 1/40
427/427 ━━━━━━━━━━━━━━━━━━━━ 110s 249ms/step - accuracy: 0.0326 - loss: 4.6423 - val_accuracy: 0.0180 - val_loss: 7.0218 - learning_rate: 1.0000e-04
Epoch 2/40
427/427 ━━━━━━━━━━━━━━━━━━━━ 105s 245ms/step - accuracy: 0.1251 - loss: 3.6344 - val_accuracy: 0.1870 - val_loss: 3.2590 - learning_rate: 1.0000e-04
Epoch 3/40
427/427 ━━━━━━━━━━━━━━━━━━━━ 106s 247ms/step - accuracy: 0.3060 - loss: 2.6889 - val_accuracy: 0.4846 - val_loss: 1.9724 - learning_rate: 1.0000e-04
Epoch 4/40
427/427 ━━━━━━━━━━━━━━━━━━━━ 105s 245ms/step - accuracy: 0.4754 - loss: 2.0498 - val_accuracy: 0.6826 - val_loss: 1.3843 - learning_rate: 1.0000e-04
Epoch 5/40
427/427 ━━━━━━━━━━━━━━━━━━━━ 104s 244ms/step - accuracy: 0.6053 - loss: 1.6545 - val_accuracy: 0.7309 - val_loss: 1.2310 - learning_rate: 1.0000e-04
Epoch 6/40
427/427 ━━━━━━━━━━━━━━━━━━━━ 105s 245ms/step - accuracy: 0.6700 - loss: 1.4379 - val_accuracy: 0.7914 - val_loss: 1.0972 - learning_rate: 1.0000e-04
Epoch 7/40
427/427 ━━━━━━━━━━━━━━━━━━━━ 

In [15]:
# Make predictions on test_ds
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))              # Convert softmax to class index
    y_true.extend(np.argmax(labels.numpy(), axis=1))     # Convert one-hot to index


Classification report

In [16]:
# Get the human-readable class labels inferred from the test directory
# (image_dataset_from_directory sorts subfolder names alphabetically).
class_names = test_ds.class_names
# Print per-class precision, recall, F1, and support, plus macro/weighted averages.
# y_true: ground-truth class indices; y_pred: predicted class indices.
# target_names maps numeric indices → label strings for a readable report.
print(classification_report(y_true, y_pred, target_names=class_names))


              precision    recall  f1-score   support

           0       0.33      0.45      0.38        11
           1       0.53      0.91      0.67        11
           2       0.92      1.00      0.96        11
           3       1.00      1.00      1.00        11
           4       1.00      1.00      1.00        11
           5       1.00      0.91      0.95        11
           6       1.00      1.00      1.00        11
           7       1.00      0.91      0.95        11
           8       1.00      1.00      1.00        11
           9       0.91      0.91      0.91        11
      A_caps       1.00      1.00      1.00        11
      B_caps       1.00      1.00      1.00        11
      C_caps       0.69      0.82      0.75        11
      D_caps       1.00      1.00      1.00        11
      E_caps       1.00      1.00      1.00        11
      F_caps       1.00      0.82      0.90        11
      G_caps       1.00      1.00      1.00        11
      H_caps       1.00    

Save model

In [28]:
# --- Mount Drive & set save folder ---
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

BASES = [Path('/content/drive/MyDrive'), Path('/content/drive/My Drive')]
DRIVE_ROOT = next((b for b in BASES if b.exists()), None)
assert DRIVE_ROOT is not None, "Can't find Google Drive mount."
SAVE_DIR = DRIVE_ROOT / "hey"
SAVE_DIR.mkdir(parents=True, exist_ok=True)
print("Saving to:", SAVE_DIR)

# --- Save the Keras model (recommended) ---
# Full model (architecture + weights + optimizer state)
model_path = SAVE_DIR / "best_model.keras"
model.save(model_path)
print("Model saved ->", model_path)

# Also save just the weights
# ---- Save just the weights ----
weights_path = SAVE_DIR / "best_model.weights.h5"  # must end with ".weights.h5"
model.save_weights(str(weights_path))
print("Weights saved ->", weights_path)


# ---Save training history for your report/plots ---
import pandas as pd, json, tensorflow as tf
hist_path = SAVE_DIR / "training_history.csv"
pd.DataFrame(history.history).to_csv(hist_path, index=False)
print("History saved ->", hist_path)

# ---Save class names (if available from your datasets) ---
class_names = None
for ds in [locals().get("validate_ds"), locals().get("augmented_ds")]:
    if hasattr(ds, "class_names"):
        class_names = ds.class_names
        break

if class_names:
    with open(SAVE_DIR / "class_names.json", "w") as f:
        json.dump(class_names, f, indent=2)
    print("Class names saved ->", SAVE_DIR / "class_names.json")
else:
    print("No class_names attribute found on your datasets; skipping class_names.json")

# ---Save minimal preprocess config (adjust as needed) ---
img_h = getattr(model.input_shape, "__getitem__", lambda x: None)(1)
img_w = getattr(model.input_shape, "__getitem__", lambda x: None)(2)
pre_cfg = {
    "img_size": [img_h, img_w] if img_h and img_w else [128, 128],  # set your true size if known
    "scale": 255,
    "color_mode": "grayscale" if (len(model.input_shape) >= 4 and model.input_shape[-1] == 1) else "rgb",
    "label_smoothing": 0.05
}
with open(SAVE_DIR / "preprocess.json", "w") as f:
    json.dump(pre_cfg, f, indent=2)
print("Preprocess config saved ->", SAVE_DIR / "preprocess.json")

# ---Quick sanity check: list saved files with sizes ---
for p in SAVE_DIR.iterdir():
    if p.is_file():
        print(f"{p.name:25s}  {(p.stat().st_size/1e6):6.2f} MB")


Mounted at /content/drive
Saving to: /content/drive/MyDrive/hey
Model saved -> /content/drive/MyDrive/hey/best_model.keras
Weights saved -> /content/drive/MyDrive/hey/best_model.weights.h5
History saved -> /content/drive/MyDrive/hey/training_history.csv
Class names saved -> /content/drive/MyDrive/hey/class_names.json
Preprocess config saved -> /content/drive/MyDrive/hey/preprocess.json
Standalone CNN Model with best parameters.ipynb    0.10 MB
bleh                         0.02 MB
best_model.keras            60.33 MB
best_model.weights.h5       60.30 MB
training_history.csv         0.00 MB
class_names.json             0.00 MB
preprocess.json              0.00 MB
